In [3]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

# Leer el archivo CSV en un DataFrame
df_transporte = pd.read_csv('Transporte_ok.csv')

# Configuración de la conexión a la base de datos
config = {
    'user': 'root',
    'password': 'AlumnaAdalab',
    'host': '127.0.0.1',
    'database': 'TransporteDB'
}

cnx = None
cursor = None

try:
    # Conectar a la base de datos
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()

    # Insertar datos en las tablas de referencia
    def insert_reference_table(table, column, value):
        query = f"INSERT IGNORE INTO {table} ({column}) VALUES (%s)"
        cursor.execute(query, (value,))
        return cursor.lastrowid

    # Insertar datos en las tablas de referencia (con chequeo)
    for index, row in df_transporte.iterrows():
        # Insertar Tipo_Transporte
        cursor.execute("SELECT ID FROM Transportes WHERE Tipo = %s", (row['Tipo_Transporte'],))
        tipo_row = cursor.fetchone()
        if tipo_row:
            tipo_id = tipo_row[0]
        else:
            tipo_id = insert_reference_table('Transportes', 'Tipo', row['Tipo_Transporte'])
        
        # Insertar Region
        cursor.execute("SELECT ID FROM Regiones WHERE Region = %s", (row['Region'],))
        region_row = cursor.fetchone()
        if region_row:
            region_id = region_row[0]
        else:
            region_id = insert_reference_table('Regiones', 'Region', row['Region'])
        
        # Insertar Dia_Semana
        cursor.execute("SELECT ID FROM Dias_Semana WHERE Dia = %s", (row['Dia_Semana'],))
        dia_semana_row = cursor.fetchone()
        if dia_semana_row:
            dia_semana_id = dia_semana_row[0]
        else:
            dia_semana_id = insert_reference_table('Dias_Semana', 'Dia', row['Dia_Semana'])
        
        # Insertar Origen
        cursor.execute("SELECT ID FROM Origenes WHERE Origen = %s", (row['Origen'],))
        origen_row = cursor.fetchone()
        if origen_row:
            origen_id = origen_row[0]
        else:
            origen_id = insert_reference_table('Origenes', 'Origen', row['Origen'])
        
        # Insertar Destino
        cursor.execute("SELECT ID FROM Destinos WHERE Destino = %s", (row['Destino'],))
        destino_row = cursor.fetchone()
        if destino_row:
            destino_id = destino_row[0]
        else:
            destino_id = insert_reference_table('Destinos', 'Destino', row['Destino'])
        
        # Insertar datos en la tabla Viajes
        add_viaje = ("INSERT INTO Viajes "
                     "(ID_Viaje, Fecha, Ruta, Numero_Pasajeros, Duracion_Viaje_Minutos, Retraso_Minutos, Tipo_Transporte_ID, Region_ID, Dia_Semana_ID, Origen_ID, Destino_ID) "
                     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
        
        data_viaje = (
            row['ID_Viaje'],
            row['Fecha'],
            row['Ruta'],
            row['Numero_Pasajeros'],
            row['Duracion_Viaje_Minutos'],
            row['Retraso_Minutos'],
            tipo_id,
            region_id,
            dia_semana_id,
            origen_id,
            destino_id
        )
        
        try:
            cursor.execute(add_viaje, data_viaje)
        except mysql.connector.Error as err:
            print(f"Error al insertar la fila {index}: {err}")
    
    # Confirmar los cambios en la base de datos
    cnx.commit()

except mysql.connector.Error as err:
    print(f"Error de conexión a MySQL: {err}")

finally:
    # Cerrar el cursor y la conexión si están definidos
    if cursor is not None:
        cursor.close()
    if cnx is not None:
        cnx.close()
